### Importing packages

In [10]:
%matplotlib inline

import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

import warnings
warnings.filterwarnings('ignore')

###  Load the pretrained Network

In [11]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

###  Freeze the layers

In [12]:
for layer in model_vgg16_conv.layers:
    layer.trainable = False

###  Training parameters

In [16]:
img_width, img_height = 150, 150
train_data_dir = 'TRAINING'
val_data_dir = 'TESTING'
model_weights_file = 'vgg16-xfer-weights.h5'
nb_train_samples = 4
nb_val_samples = 4
nb_epochs = 5

###  Build a classification model on top of Base Network

In [17]:
input = Input(shape=(img_width, img_height, 3))
output_vgg16_conv = model_vgg16_conv(input)
x = Flatten()(output_vgg16_conv)
x = Dense(64, activation='relu')(x)
x = Dense(2, activation='softmax')(x)
model = Model(input=input, output=x)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

###  Dataset Preparation

In [18]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=4, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                        batch_size=4,class_mode='categorical')

Found 4 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


###  Training

In [19]:
callbacks = [ModelCheckpoint(model_weights_file, monitor='val_acc', save_best_only=True)]

history = model.fit_generator( train_generator, callbacks = callbacks, samples_per_epoch=nb_train_samples, 
                              nb_epoch=nb_epochs, validation_data=validation_generator, nb_val_samples=nb_val_samples)

print('Training Completed!')

Epoch 1/5
1/1 [==============================] - 28s - loss: 0.7669 - acc: 0.5000 - val_loss: 1.9878 - val_acc: 0.5000
Epoch 2/5
1/1 [==============================] - 14s - loss: 1.8849 - acc: 0.5000 - val_loss: 4.6661 - val_acc: 0.5000
Epoch 3/5
1/1 [==============================] - 23s - loss: 2.5775 - acc: 0.5000 - val_loss: 0.0984 - val_acc: 1.0000
Epoch 4/5
1/1 [==============================] - 15s - loss: 0.0100 - acc: 1.0000 - val_loss: 0.1324 - val_acc: 1.0000
Epoch 5/5
1/1 [==============================] - 14s - loss: 0.0060 - acc: 1.0000 - val_loss: 0.1076 - val_acc: 1.0000
Training Completed!


###  Test the model

In [24]:
img_path = 'chiti.jpg'
label = ['Man','Robo']
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
ind = np.where(features == 1)[1][0]
print('Predicted Array:',features)
print('Predicted Label:',label[ind])

IndexError: index 0 is out of bounds for axis 0 with size 0

### Models for image classification with weights trained on ImageNet
* Xception
* VGG16
* VGG19
* ResNet50
* InceptionV3
* InceptionResNetV2
* MobileNet
* DenseNet
* NASNet